In [ ]:
from data_generator import StudyParameters
from experiment1_solution_quality import DynamicProgramming
import logging
from pprint import pformat

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def create_simple_test_instance():
    """Create test instance with specified parameters."""
    params = StudyParameters(
        T=2,          # 2 booking periods
        N=3,          # 3-day horizon
        C=5,          # 5 rooms capacity
        price_min=50, # Minimum price
        price_max=150,# Maximum price
        alpha=0.1,    # Not used in DP
        beta=0.1      # Not used in DP
    )
    
    # Generate booking classes
    booking_classes = []
    for arrival in range(1, 4):
        for departure in range(arrival, 4):
            booking_classes.append((arrival, departure))
    
    # Uniform arrival probabilities (0.15)
    arrival_probs = {}
    for t in range(1, 3):
        arrival_probs[t] = {bc: 0.15 for bc in booking_classes}
    
    # Uniform price sensitivity (1/max_price)
    epsilon = 1/params.price_max
    price_sensitivity = {bc: epsilon for bc in booking_classes}
    
    return {
        'parameters': params,
        'booking_classes': booking_classes,
        'arrival_probabilities': arrival_probs,
        'reservation_price_params': price_sensitivity
    }

In [ ]:
def main():
    test_instance = create_simple_test_instance()
    
    # Print instance details
    logger.info("Test Instance Details:")
    logger.info(f"Time Periods (T): {test_instance['parameters'].T}")
    logger.info(f"Service Horizon (N): {test_instance['parameters'].N}")
    logger.info(f"Room Capacity (C): {test_instance['parameters'].C}")
    logger.info(f"Price Range: [{test_instance['parameters'].price_min}, {test_instance['parameters'].price_max}]")
    
    # Solve using Dynamic Programming
    dp = DynamicProgramming(test_instance)
    results, optimal_value = dp.solve()
    
    # Print formatted results for states with capacity 5
    logger.info("\nValue Function for States with Capacity 5:")
    logger.info(pformat(results))
    
    logger.info(f"\nOptimal Value: {optimal_value:.4f}")

In [ ]:
if __name__ == "__main__":
    main()